# KNN Test 1

## Data Cleaning

In [165]:
import pandas as pd
import seaborn as sb
import numpy as np
import sklearn as sk
import matplotlib as plt
%matplotlib inline

In [166]:
df = pd.read_csv('../Event Recordings/vol2.csv')

In [167]:
df.head()

,Timestamp,X,Y,BTN_TOUCH,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER
0,0.000000,326,1710,DOWN,10,-420,3537,24,0
1,0.066941,324,-420,NaN,-420,-420,-420,-420,0
2,0.066941,332,264,NaN,10,-420,3538,26,1
3,0.075072,321,1711,HELD,-420,-420,-420,-420,0
4,0.082513,318,1711,HELD,-420,-420,-420,-420,0


**Sort Data by finger**

In [168]:
# Sort values by finger first, then timestamp
df = df.sort_values(['FINGER','Timestamp'])

**Convert String data to Numerical data**

In [169]:
df = pd.get_dummies(df,columns=['BTN_TOUCH'],drop_first=True)

In [170]:
df.head()

,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,BTN_TOUCH_UP
0,0.000000,326,1710,10,-420,3537,24,0,0,0
1,0.066941,324,-420,-420,-420,-420,-420,0,0,0
3,0.075072,321,1711,-420,-420,-420,-420,0,1,0
4,0.082513,318,1711,-420,-420,-420,-420,0,1,0
5,0.091629,316,1711,-420,-420,-420,-420,0,1,0


**Handle missing data**

Either Remove Rows, Remove Column, or Fill with Data

**Does data need to be scaled?**

**Does data need to be standardized?**

**Does data need to be normalized?**

## Feature Extraction

In [171]:
df.insert(len(df.columns), "X_Speed", 0)
df.insert(len(df.columns), "X_Acceleration", 0)
df.insert(len(df.columns), "Y_Speed", 0)
df.insert(len(df.columns), "Y_Acceleration", 0)
df.insert(len(df.columns), "Speed", 0)
df.insert(len(df.columns), "Acceleration", 0)
df.insert(len(df.columns), "Jerk", 0)
df.insert(len(df.columns), "Ang_V", 0)
df.insert(len(df.columns), "Path_Tangent", 0)

In [172]:
df.head()

,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,BTN_TOUCH_UP,X_Speed,X_Acceleration,Y_Speed,Y_Acceleration,Speed,Acceleration,Jerk,Ang_V,Path_Tangent
0,0.000000,326,1710,10,-420,3537,24,0,0,0,0,0,0,0,0,0,0,0,0
1,0.066941,324,-420,-420,-420,-420,-420,0,0,0,0,0,0,0,0,0,0,0,0
3,0.075072,321,1711,-420,-420,-420,-420,0,1,0,0,0,0,0,0,0,0,0,0
4,0.082513,318,1711,-420,-420,-420,-420,0,1,0,0,0,0,0,0,0,0,0,0
5,0.091629,316,1711,-420,-420,-420,-420,0,1,0,0,0,0,0,0,0,0,0,0


*I am really confused on these calculations*

From my understanding we need to divide up our data into "gestures" comprised of either x mouse events or from x time periods.

What is the .shift() function really doing?

What do the gestures look like? The difference between the values, the average values, all of the values? And how do we extract features from that?

In [173]:
df['X_Speed'] = (df.X - df.X.shift(1)) / (df.Timestamp - df.Timestamp.shift(1))
df['Y_Speed'] = (df.Y - df.Y.shift(1)) / (df.Timestamp - df.Timestamp.shift(1))
df['Speed'] = np.sqrt((df.X_Speed ** 2) + (df.Y_Speed ** 2))
df['X_Acceleration'] = (df.X_Speed - df.X_Speed.shift(1)) / (df.Timestamp - df.Timestamp.shift(1))
df['Y_Acceleration'] = (df.Y_Speed - df.Y_Speed.shift(1)) / (df.Timestamp - df.Timestamp.shift(1))
df['Acceleration'] = (df.Speed - df.Speed.shift(1)) / (df.Timestamp - df.Timestamp.shift(1))
df['Jerk'] = (df.Acceleration - df.Acceleration.shift(1)) / (df.Timestamp - df.Timestamp.shift(1))
df['Path_Tangent'] = np.arctan2((df.Y - df.Y.shift(1)), (df.X - df.X.shift(1)))
df['Ang_V'] = (df.Path_Tangent - df.Path_Tangent.shift(1)) / (df.Timestamp - df.Timestamp.shift(1))

In [174]:
df.head()

,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,BTN_TOUCH_UP,X_Speed,X_Acceleration,Y_Speed,Y_Acceleration,Speed,Acceleration,Jerk,Ang_V,Path_Tangent
0,0.000000,326,1710,10,-420,3537,24,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.066941,324,-420,-420,-420,-420,-420,0,0,0,-29.877056,NaN,-31819.064549,NaN,31819.078576,NaN,NaN,NaN,-1.571735
3,0.075072,321,1711,-420,-420,-420,-420,0,1,0,-368.958308,-41702.281627,262083.384620,3.614592e+07,262083.644328,2.831934e+07,NaN,386.660855,1.572204
4,0.082513,318,1711,-420,-420,-420,-420,0,1,0,-403.171617,-4597.945015,0.000000,-3.522153e+07,403.171617,-3.516738e+07,-8.532015e+09,210.910971,3.141593
5,0.091629,316,1711,-420,-420,-420,-420,0,1,0,-219.394471,20159.844818,0.000000,0.000000e+00,219.394471,-2.015984e+04,3.855553e+09,0.000000,3.141593


In [175]:
# df.to_csv('OUT.csv')
# to check output is correct

**Drop missing or NaN values**

In [176]:
df.dropna(inplace=True)

## Add user column and save to csv for later

In [177]:
#df['User'] = 0

In [178]:
#df.to_csv('User1features.csv')

### Do this^ all for another user

So we are training to recognize gestures? Not series of gestures.

## Train Test Split

In [179]:
from sklearn.model_selection import train_test_split

In [180]:
User0 = pd.read_csv('User0features.csv')
User1 = pd.read_csv('User1features.csv')

In [181]:
User1

,Unnamed: 0,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,...,X_Speed,X_Acceleration,Y_Speed,Y_Acceleration,Speed,Acceleration,Jerk,Ang_V,Path_Tangent,User
0,4,0.082513,318,1711,-420,-420,-420,-420,0,1,...,-403.171617,-4597.945015,0.000000,-3.522153e+07,403.171617,-3.516738e+07,-8.532015e+09,210.910971,3.141593,1
1,5,0.091629,316,1711,-420,-420,-420,-420,0,1,...,-219.394471,20159.844818,0.000000,0.000000e+00,219.394471,-2.015984e+04,3.855553e+09,0.000000,3.141593,1
2,6,0.099507,311,1710,-420,-420,-420,-420,0,1,...,-634.678852,-52714.442896,-126.935770,-1.611269e+04,647.247971,5.430991e+04,9.452876e+06,-772.504411,-2.944197,1
3,8,0.108328,306,1710,-420,-420,-420,-420,0,1,...,-566.829157,7691.837088,0.000000,1.439018e+04,566.829157,-9.116746e+03,-7.190416e+06,689.920615,3.141593,1
4,10,0.116439,300,1709,-420,-420,-420,-420,0,1,...,-739.736161,-21317.593807,-123.289360,-1.520027e+04,749.939900,2.257561e+04,3.907330e+06,-754.288821,-2.976444,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39897,39896,900.476998,427,335,11,-420,4423,24,1,1,...,-523.012552,-20618.920869,0.000000,-1.592220e+04,523.012552,1.803510e+04,3.788121e+06,42.069895,3.141593,1
39898,39898,900.486026,425,336,11,-420,4423,24,1,1,...,-221.533008,33393.835151,110.766504,1.226922e+04,247.681433,-3.049747e+04,-5.375783e+06,-51.356625,2.677945,1
39899,39900,900.494168,422,336,11,-420,4423,24,1,1,...,-368.459838,-18045.545226,0.000000,-1.360434e+04,368.459838,1.483400e+04,5.567608e+06,56.945174,3.141593,1
39900,39902,900.502887,419,336,11,-420,4423,24,1,1,...,-344.076156,2796.614538,0.000000,0.000000e+00,344.076156,-2.796615e+03,-2.022091e+06,0.000000,3.141593,1


In [182]:
final_feature_set = pd.concat([User0, User1], ignore_index=True)

In [183]:
final_feature_set

,Unnamed: 0,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,...,X_Speed,X_Acceleration,Y_Speed,Y_Acceleration,Speed,Acceleration,Jerk,Ang_V,Path_Tangent,User
0,3,0.049881,344,1411,11,-420,2526,38,0,1,...,-386.001029,-36025.503880,386.001029,22385.409554,545.887891,39737.584434,3.060251e+06,41.398682,2.356194,0
1,4,0.058581,340,1416,11,-420,2526,38,0,1,...,-459.770115,-8479.205245,574.712644,21690.990156,735.991292,21850.965622,-2.055933e+06,-12.719221,2.245537,0
2,5,0.066807,335,1420,11,-420,2526,38,0,1,...,-607.828835,-17998.871928,486.263068,-10752.440473,778.400710,5155.533432,-2.029593e+06,26.904260,2.466852,0
3,6,0.075273,330,1424,11,-420,2526,38,0,1,...,-590.597685,2035.335519,472.478148,-1628.268415,756.334070,-2606.501238,-9.168479e+05,0.000000,2.466852,0
4,7,0.083722,324,1428,11,-420,2526,38,0,1,...,-710.143212,-14149.074119,473.428808,112.517478,853.485921,11498.621289,1.669443e+06,10.266107,2.553590,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99459,39896,900.476998,427,335,11,-420,4423,24,1,1,...,-523.012552,-20618.920869,0.000000,-15922.203851,523.012552,18035.102883,3.788121e+06,42.069895,3.141593,1
99460,39898,900.486026,425,336,11,-420,4423,24,1,1,...,-221.533008,33393.835151,110.766504,12269.218464,247.681433,-30497.465562,-5.375783e+06,-51.356625,2.677945,1
99461,39900,900.494168,422,336,11,-420,4423,24,1,1,...,-368.459838,-18045.545226,0.000000,-13604.336062,368.459838,14833.997126,5.567608e+06,56.945174,3.141593,1
99462,39902,900.502887,419,336,11,-420,4423,24,1,1,...,-344.076156,2796.614538,0.000000,0.000000,344.076156,-2796.614538,-2.022091e+06,0.000000,3.141593,1


In [184]:
X = final_feature_set
y = final_feature_set['User']

In [185]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## Using KNN

In [190]:
from sklearn.neighbors import KNeighborsClassifier

In [196]:
knn = KNeighborsClassifier(n_neighbors=10)
# n_neighbors should be adjusted to best value

In [197]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

## Predict and Evaluate

In [198]:
pred = knn.predict(X_test)

In [199]:
from sklearn.metrics import classification_report, confusion_matrix

In [200]:
print(confusion_matrix(y_test, pred))

[[14402  3578]
 [ 7491  4369]]


\[TP, FP
TN, FN]